# VacationPy

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import gmaps
import time

# Google developer API key
from config import key

# Access maps with unique API key
gmaps.configure(api_key=key)

In [ ]:
key

In [26]:
# Load WeatherPy.csv
cleanCity_df = pd.read_csv("../WeatherPy/city_weather.csv")
cleanCity_df.head(30)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,75.36,69,20,12.66,TO,1627954382
1,1,ahuimanu,21.4447,-157.8378,87.51,56,1,14.97,US,1627954518
2,3,anadyr,64.7500,177.4833,50.50,81,90,17.00,RU,1627954386
3,5,arraial do cabo,-22.9661,-42.0278,67.48,82,2,17.81,BR,1627954383
4,6,aguadulce,8.2500,-80.5500,78.13,86,100,4.85,PA,1627954525
5,7,souillac,-20.5167,57.5167,59.38,88,20,8.05,MU,1627954431
6,8,birnin kebbi,12.4539,4.1975,78.76,76,100,6.31,NG,1627954528
7,9,bathsheba,13.2167,-59.5167,81.21,78,20,10.36,BB,1627954370
8,10,roald,62.5833,6.1333,53.01,81,75,8.05,NO,1627954377
9,11,new norfolk,-42.7826,147.0587,52.75,73,97,1.99,AU,1627954376


## Humidity Heatmap

In [7]:
locations = cleanCity_df[["Lat","Lng"]]
humidity = cleanCity_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# Ideal weather conditions

humidity = cleanCity_df["Humidity"]<55
temp = ((cleanCity_df["Max Temp"]<=86) & (cleanCity_df["Max Temp"]>=70))
winds = cleanCity_df["Wind Speed"]<10

idealCities_df = cleanCity_df.loc[(temp&humidity&winds),:].reset_index()
# idealCities_df



# Locate first hotel within 5000m of ideal cities

# Creating hotel list for ideal cities
hotelList_df = idealCities_df[["City", "Country","Lat","Lng"]]
hotelList_df["Closest Hotel"] = ""



<ipython-input-50-e73e86d8cb42>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelList_df["Closest Hotel"] = ""


In [52]:
# hotelList_df.head()

,City,Country,Lat,Lng,Closest Hotel
0,atambua,ID,-9.1061,124.8925,
1,aksha,RU,50.2814,113.2867,
2,gympie,AU,-26.1898,152.6650,
3,hami,CN,42.8000,93.4500,
4,jiuquan,CN,39.7432,98.5174,


In [54]:
# Call API data

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000,
          "type": "lodging",
          "key": key
         }

for index, row in idealCities_df.iterrows():
    
    # Name of City
    cityName = row["City"]
          
    # Add lat/lon to params dict
    Lat = row["Lat"]
    Lng = row["Lng"]       
    coordinates = f"{Lat},{Lng}"
    params["location"] = coordinates     

    # URL assembly an d API request
    print(f"Retrieving Results for: {cityName}.")
    time.sleep(1)
    response = requests.get(base_url,params=params).json()
    
    # Store results
    results = response["results"]
          
    try:
        print(f"Closest hotel in {cityName} to ({coordinates}) is {results[0]['name']}.")
        hotelList_df.loc[index, "Closest Hotel"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping")
          
    print("---------------")
          
    

Retrieving Results for: atambua.
Closest hotel in atambua to (-9.1061,124.8925) is Pastori 1 Jemaat Polycarpus.
---------------
Retrieving Results for: aksha.


C:\Users\wodd\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel in aksha to (50.2814,113.2867) is Vizit.
---------------
Retrieving Results for: gympie.
Closest hotel in gympie to (-26.1898,152.665) is Mary River Motor Inn.
---------------
Retrieving Results for: hami.
Closest hotel in hami to (42.8,93.45) is Laoyutang Resort.
---------------
Retrieving Results for: jiuquan.
Closest hotel in jiuquan to (39.7432,98.5174) is Jiuquan Hotel.
---------------
Retrieving Results for: tuntum.
Closest hotel in tuntum to (-5.2581,-44.6489) is Casa da Dona Maria Lúcia.
---------------
Retrieving Results for: aripuana.
Missing field/result... skipping
---------------
Retrieving Results for: college.
Closest hotel in college to (64.8569,-147.8028) is Pike's Waterfront Lodge.
---------------
Retrieving Results for: nyurba.
Closest hotel in nyurba to (63.2842,118.3319) is Gostinitsa Alrosa.
---------------
Retrieving Results for: labrea.
Closest hotel in labrea to (-7.2586,-64.7981) is Hotel Novo Horizonte.
---------------
Retrieving Results for: th

In [56]:
# Export hotel list to .csv
hotelList_df.to_csv("Ideal_cities_hotel_list.csv", index=True)

In [57]:
# Clear cities without hotel in range

# hotelList_df = pd.read_csv("Ideal_cities_hotel_list.csv")
hotelsCleaned_df = hotelList_df[hotelList_df["Closest Hotel"].str.strip().astype(bool)].reset_index()

# hotelsCleaned_df.head()

,index,City,Country,Lat,Lng,Closest Hotel
0,0,atambua,ID,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
1,1,aksha,RU,50.2814,113.2867,Vizit
2,2,gympie,AU,-26.1898,152.6650,Mary River Motor Inn
3,3,hami,CN,42.8000,93.4500,Laoyutang Resort
4,4,jiuquan,CN,39.7432,98.5174,Jiuquan Hotel


In [ ]:
# Plot heatmap with added hotel markers

locations = hotelsCleaned_df[["Lat","Lng"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

# Convert bank rate to list
bank_rate = census_data_complete["Bank Count"].tolist()

# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig
Figure(layout=FigureLayout(height='420px'))

# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig
Figure(layout=FigureLayout(height='420px'))